<a href="https://colab.research.google.com/github/mlelarge/dataflowr/blob/master/CEA_EDF_INRIA/03_collaborative_filtering_full_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collaborative filtering
-----

In this example, we'll build a quick explicit feedback recommender system: that is, a model that takes into account explicit feedback signals (like ratings) to recommend new content.

We'll use an approach first made popular by the [Netflix prize](http://www.netflixprize.com/) contest: [matrix factorization](https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf). 

The basic idea is very simple:

1. Start with user-item-rating triplets, conveying the information that user _i_ gave some item _j_ rating _r_.
2. Represent both users and items as high-dimensional vectors of numbers. For example, a user could be represented by `[0.3, -1.2, 0.5]` and an item by `[1.0, -0.3, -0.6]`.
3. The representations should be chosen so that, when we multiplied together (via [dot products](https://en.wikipedia.org/wiki/Dot_product)), we can recover the original ratings.
4. The utility of the model then is derived from the fact that if we multiply the user vector of a user with the item vector of some item they _have not_ rated, we hope to obtain a predicition for the rating they would have given to it had they seen it.

![collaborative filtering](http://ampcamp.berkeley.edu/big-data-mini-course/img/matrix_factorization.png)
source:[ampcamp.berkeley](http://ampcamp.berkeley.edu/big-data-mini-course/movie-recommendation-with-mllib.html)

## 1. Preparations

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os.path as op

from zipfile import ZipFile
try:
    from urllib.request import urlretrieve
except ImportError:  # Python 2 compat
    from urllib import urlretrieve

# this line need to be changed:
data_folder = '/content/'

ML_100K_URL = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
ML_100K_FILENAME = op.join(data_folder,ML_100K_URL.rsplit('/', 1)[1])
ML_100K_FOLDER = op.join(data_folder,'ml-100k')

We start with importing a famous dataset, the [Movielens 100k dataset](https://grouplens.org/datasets/movielens/100k/). It contains 100,000 ratings (between 1 and 5) given to 1683 movies by 944 users:

In [4]:
if not op.exists(ML_100K_FILENAME):
    print('Downloading %s to %s...' % (ML_100K_URL, ML_100K_FILENAME))
    urlretrieve(ML_100K_URL, ML_100K_FILENAME)

if not op.exists(ML_100K_FOLDER):
    print('Extracting %s to %s...' % (ML_100K_FILENAME, ML_100K_FOLDER))
    ZipFile(ML_100K_FILENAME).extractall(data_folder)

Extracting /content/ml-100k.zip to /content/ml-100k...


Other datasets, see: [Movielens](https://grouplens.org/datasets/movielens/)

## 2. Data analysis and formating

[Python Data Analysis Library](http://pandas.pydata.org/)

In [5]:
import pandas as pd

all_ratings = pd.read_csv(op.join(ML_100K_FOLDER, 'u.data'), sep='\t',
                          names=["user_id", "item_id", "ratings", "timestamp"])
all_ratings.head()

,user_id,item_id,ratings,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Let's check out a few macro-stats of our dataset

In [6]:
#number of entries
len(all_ratings)

100000

In [7]:
all_ratings['ratings'].describe()

count    100000.000000
mean          3.529860
std           1.125674
min           1.000000
25%           3.000000
50%           4.000000
75%           4.000000
max           5.000000
Name: ratings, dtype: float64

In [8]:
# number of unique rating values
len(all_ratings['ratings'].unique())

5

In [9]:
all_ratings['user_id'].describe()

count    100000.00000
mean        462.48475
std         266.61442
min           1.00000
25%         254.00000
50%         447.00000
75%         682.00000
max         943.00000
Name: user_id, dtype: float64

In [10]:
# number of unique users
total_user_id = len(all_ratings['user_id'].unique())
print(total_user_id)

943


In [11]:
all_ratings['item_id'].describe()

count    100000.000000
mean        425.530130
std         330.798356
min           1.000000
25%         175.000000
50%         322.000000
75%         631.000000
max        1682.000000
Name: item_id, dtype: float64

In [12]:
# number of unique rated items
total_item_id = len(all_ratings['item_id'].unique())
print(total_item_id)

1682


For spliting the data into _train_ and _test_ we'll be using a pre-defined function from [scikit-learn](http://scikit-learn.org/stable/)

In [0]:
from sklearn.model_selection import train_test_split

ratings_train, ratings_test = train_test_split(
    all_ratings, test_size=0.2, random_state=42)

user_id_train = ratings_train['user_id']
item_id_train = ratings_train['item_id']
rating_train = ratings_train['ratings']

user_id_test = ratings_test['user_id']
item_id_test = ratings_test['item_id']
rating_test = ratings_test['ratings']

In [14]:
len(user_id_train)

80000

In [15]:
len(user_id_train.unique())

943

In [16]:
len(item_id_train.unique())

1653

We see that all the movies are not rated in the train set.

In [17]:
user_id_train.iloc[:5]

75220    807
48955    474
44966    463
13568    139
92727    621
Name: user_id, dtype: int64

In [18]:
item_id_train.iloc[:5]

75220    1411
48955     659
44966     268
13568     286
92727     751
Name: item_id, dtype: int64

In [19]:
rating_train.iloc[:5]

75220    1
48955    5
44966    4
13568    4
92727    4
Name: ratings, dtype: int64

## 3. The model

We can feed our dataset to the `FactorizationModel` class - a sklearn-like object that allows us to train and evaluate the explicit factorization models.

Internally, the model uses the `Model_dot`(class to represents users and items. It's composed of a 4 `embedding` layers:

- a `(num_users x latent_dim)` embedding layer to represent users,
- a `(num_items x latent_dim)` embedding layer to represent items,
- a `(num_users x 1)` embedding layer to represent user biases, and
- a `(num_items x 1)` embedding layer to represent item biases.

In [0]:
import torch.nn as nn
import torch

Let's generate [Embeddings](http://pytorch.org/docs/master/nn.html#embedding) for the users, _i.e._ a fixed-sized vector describing the user

In [21]:
embedding_dim = 3
embedding_user = nn.Embedding(total_user_id, embedding_dim)
input = torch.LongTensor([[1,2,4,5],[4,3,2,0]])
embedding_user(input)

tensor([[[-0.1832, -0.9930,  2.5870],
         [ 1.1143,  0.9589, -0.7250],
         [-0.3251, -1.6406, -0.3944],
         [ 0.9714, -0.1434, -0.0922]],

        [[-0.3251, -1.6406, -0.3944],
         [ 1.4056,  0.3354, -0.0237],
         [ 1.1143,  0.9589, -0.7250],
         [ 0.6052,  0.8300,  0.9265]]], grad_fn=<EmbeddingBackward>)

We will use some custom embeddings and dataloader

In [0]:
class ScaledEmbedding(nn.Embedding):
    """
    Embedding layer that initialises its values
    to using a normal variable scaled by the inverse
    of the emedding dimension.
    """

    def reset_parameters(self):
        """
        Initialize parameters.
        """

        self.weight.data.normal_(0, 1.0 / self.embedding_dim)
        if self.padding_idx is not None:
            self.weight.data[self.padding_idx].fill_(0)


class ZeroEmbedding(nn.Embedding):
    """
    Used for biases.
    """

    def reset_parameters(self):
        """
        Initialize parameters.
        """

        self.weight.data.zero_()
        if self.padding_idx is not None:
            self.weight.data[self.padding_idx].fill_(0)

In [0]:
class DotModel(nn.Module):
    
    def __init__(self,
                 num_users,
                 num_items,
                 embedding_dim=32):
        
        super(DotModel, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        self.user_embeddings = ScaledEmbedding(num_users, embedding_dim)
        self.item_embeddings = ScaledEmbedding(num_items, embedding_dim)
        self.user_biases = ZeroEmbedding(num_users, 1)
        self.item_biases = ZeroEmbedding(num_items, 1)
                
        
    def forward(self, user_ids, item_ids):
        
        user_embedding = self.user_embeddings(user_ids)
        item_embedding = self.item_embeddings(item_ids)

        user_embedding = user_embedding.squeeze()
        item_embedding = item_embedding.squeeze()

        user_bias = self.user_biases(user_ids).squeeze()
        item_bias = self.item_biases(item_ids).squeeze()

        dot = (user_embedding * item_embedding).sum(1)

        return dot + user_bias + item_bias


In [0]:
def gpu(tensor, gpu=False):

    if gpu:
        return tensor.cuda()
    else:
        return tensor


In [0]:
def shuffle(*arrays):

    random_state = np.random.RandomState()
    shuffle_indices = np.arange(len(arrays[0]))
    random_state.shuffle(shuffle_indices)

    if len(arrays) == 1:
        return arrays[0][shuffle_indices]
    else:
        return tuple(x[shuffle_indices] for x in arrays)

In [0]:
def minibatch(batch_size, *tensors):

    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i:i + batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i:i + batch_size] for x in tensors)


def regression_loss(predicted_ratings, observed_ratings):
    return ((observed_ratings - predicted_ratings) ** 2).mean()

In [0]:
import imp
import numpy as np

import torch.optim as optim

class FactorizationModel(object):
    
    def __init__(self,
                 embedding_dim=32,
                 n_iter=10,
                 batch_size=256,
                 l2=0.0,
                 learning_rate=1e-2,
                 use_cuda=False,
                 net=None,
                 num_users=None,
                 num_items=None, 
                 random_state=None):
        
        self._embedding_dim = embedding_dim
        self._n_iter = n_iter
        self._learning_rate = learning_rate
        self._batch_size = batch_size
        self._l2 = l2
        self._use_cuda = use_cuda
        
        self._num_users = num_users
        self._num_items = num_items
        self._net = net
        self._optimizer = None
        self._loss_func = None
        self._random_state = random_state or np.random.RandomState()
             
        
    def _initialize(self):
        if self._net is None:
            self._net = gpu(DotModel(self._num_users, self._num_items, self._embedding_dim),self._use_cuda)
        
        self._optimizer = optim.Adam(
                self._net.parameters(),
                lr=self._learning_rate,
                weight_decay=self._l2
            )
        
        self._loss_func = regression_loss
    
    @property
    def _initialized(self):
        return self._optimizer is not None
    
    def __repr__(self):
        return _repr_model(self)
    
    def fit(self, user_ids, item_ids, ratings, verbose=True):
        
        user_ids = user_ids.astype(np.int64)
        item_ids = item_ids.astype(np.int64)
        
        if not self._initialized:
            self._initialize()
            
        for epoch_num in range(self._n_iter):
            users, items, ratingss = shuffle(user_ids,
                                            item_ids,
                                            ratings)

            user_ids_tensor = gpu(torch.from_numpy(users),
                                  self._use_cuda)
            item_ids_tensor = gpu(torch.from_numpy(items),
                                  self._use_cuda)
            ratings_tensor = gpu(torch.from_numpy(ratingss),
                                 self._use_cuda)
            epoch_loss = 0.0

            for (minibatch_num,
                 (batch_user,
                  batch_item,
                  batch_rating)) in enumerate(minibatch(self._batch_size,
                                                         user_ids_tensor,
                                                         item_ids_tensor,
                                                         ratings_tensor)):
                
                
        
                predictions = self._net(batch_user, batch_item)

                self._optimizer.zero_grad()
                
                loss = self._loss_func(predictions,batch_rating)
                
                epoch_loss = epoch_loss + loss.data.item()
                
                loss.backward()
                self._optimizer.step()
                
            
            epoch_loss = epoch_loss / (minibatch_num + 1)

            if verbose:
                print('Epoch {}: loss {}'.format(epoch_num, epoch_loss))
        
            if np.isnan(epoch_loss) or epoch_loss == 0.0:
                raise ValueError('Degenerate epoch loss: {}'
                                 .format(epoch_loss))
    
    
    def test(self,user_ids, item_ids, ratings):
        self._net.train(False)
        user_ids = user_ids.astype(np.int64)
        item_ids = item_ids.astype(np.int64)
        
        user_ids_tensor = gpu(torch.from_numpy(user_ids),
                                  self._use_cuda)
        item_ids_tensor = gpu(torch.from_numpy(item_ids),
                                  self._use_cuda)
        ratings_tensor = gpu(torch.from_numpy(ratings),
                                 self._use_cuda)
               
        predictions = self._net(user_ids_tensor, item_ids_tensor)
        
        loss = self._loss_func(ratings_tensor, predictions)
        return loss.data.item()
      
    def predict(self,user_ids, item_ids):
        self._net.train(False)
        user_ids = user_ids.astype(np.int64)
        item_ids = item_ids.astype(np.int64)
        
        user_ids_tensor = gpu(torch.from_numpy(user_ids),
                                  self._use_cuda)
        item_ids_tensor = gpu(torch.from_numpy(item_ids),
                                  self._use_cuda)
               
        predictions = self._net(user_ids_tensor, item_ids_tensor)
        return predictions.data
      

In [0]:
model = FactorizationModel(embedding_dim=128,  # latent dimensionality
                                   n_iter=10,  # number of epochs of training
                                   batch_size=1024,  # minibatch size
                                   learning_rate=1e-3,
                                   l2=1e-9,  # strength of L2 regularization
                                   use_cuda=torch.cuda.is_available(),
                                   num_users=total_user_id+1,
                                   num_items=total_item_id+1)

In [0]:
user_ids_train_np = user_id_train.values.astype(np.int32)
item_ids_train_np = item_id_train.values.astype(np.int32)
ratings_train_np = rating_train.values.astype(np.float32)

In [73]:
model.fit(user_ids_train_np, item_ids_train_np, ratings_train_np)

Epoch 0: loss 13.094120001491111
Epoch 1: loss 7.326523219482808
Epoch 2: loss 1.7837381996685946
Epoch 3: loss 1.0725114843513393
Epoch 4: loss 0.9416675303555742
Epoch 5: loss 0.8899725079536438
Epoch 6: loss 0.8588055575950236
Epoch 7: loss 0.8409514283832116
Epoch 8: loss 0.822383515442474
Epoch 9: loss 0.8128750746763205


In [74]:
print(model._net)

DotModel(
  (user_embeddings): ScaledEmbedding(944, 128)
  (item_embeddings): ScaledEmbedding(1683, 128)
  (user_biases): ZeroEmbedding(944, 1)
  (item_biases): ZeroEmbedding(1683, 1)
)


In [75]:
user_ids_test_np = user_id_test.values.astype(np.int64)
item_ids_test_np = item_id_test.values.astype(np.int64)
ratings_test_np = rating_test.values.astype(np.float32)
model.test(user_ids_test_np, item_ids_test_np, ratings_test_np  )

0.8743864297866821

In [77]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

test_preds = model.predict(user_ids_test_np, item_ids_test_np)
print("Final test RMSE: %0.3f" % np.sqrt(mean_squared_error(test_preds, rating_test)))
print("Final test MAE: %0.3f" % mean_absolute_error(test_preds, rating_test))

tensor([3.6889, 3.7649, 3.6159,  ..., 3.8305, 3.5982, 3.1344])
Final test RMSE: 0.935
Final test MAE: 0.735


You can compare with [Surprise](https://github.com/NicolasHug/Surprise)

## 4. Best and worst movies

Getting the name of the movies (there must be a better way, please provide alternate solutions!)

In [78]:
df = pd.read_csv(op.join(ML_100K_FOLDER, 'u.item'), sep='|', names=['item_id', 'item_name','date','','','','','','','','','','','','','','','','','','','','',''],encoding = "ISO-8859-1")
movies_names = df.loc[:,['item_id', 'item_name']]
movies_names = movies_names.set_index(['item_id'])
movies_names.head()

/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


,item_name
item_id,
1,Toy Story (1995)
2,GoldenEye (1995)
3,Four Rooms (1995)
4,Get Shorty (1995)
5,Copycat (1995)


In [0]:
item_bias_np = model._net.item_biases.weight.data.cpu().numpy()

In [0]:
movies_names['biases'] = pd.Series(item_bias_np[1:].T[0], index=movies_names.index)

In [81]:
movies_names.head()

,item_name,biases
item_id,,
1,Toy Story (1995),0.132117
2,GoldenEye (1995),0.101034
3,Four Rooms (1995),0.100352
4,Get Shorty (1995),0.115437
5,Copycat (1995),0.107443


In [82]:
movies_names.shape

(1682, 2)

In [83]:
indices_item_train = np.sort(item_id_train.unique())
movies_names = movies_names.loc[indices_item_train]
movies_names.shape

(1653, 2)

In [0]:
movies_names = movies_names.sort_values(ascending=False,by=['biases'])

Best movies

In [85]:
movies_names.head(10)

,item_name,biases
item_id,,
302,L.A. Confidential (1997),0.159430
313,Titanic (1997),0.155881
272,Good Will Hunting (1997),0.155545
315,Apt Pupil (1998),0.155507
603,Rear Window (1954),0.148910
316,As Good As It Gets (1997),0.145957
269,"Full Monty, The (1997)",0.145842
50,Star Wars (1977),0.145774
258,Contact (1997),0.144823


Worse movies

In [86]:
movies_names.tail(10)

,item_name,biases
item_id,,
1618,King of New York (1990),0.029075
1661,"New Age, The (1994)",0.028254
1621,Butterfly Kiss (1995),0.028239
1491,Tough and Deadly (1995),0.027506
1654,Chairman of the Board (1998),0.027342
1494,"Mostro, Il (1994)",0.026660
1486,Girl in the Cadillac (1995),0.026268
1146,Calendar Girl (1993),0.024206
599,Police Story 4: Project S (Chao ji ji hua) (1993),0.023872


## 5. SPOTLIGHT

The code written above is a simplified version of [SPOTLIGHT](https://github.com/maciejkula/spotlight)

Once you installed it with: `conda install -c maciejkula -c pytorch spotlight=0.1.5`, you can compare the results...